In [1]:
# Apply custon style to notebook
from IPython.core.display import HTML
import pathlib
styles_path = pathlib.Path(pathlib.Path().absolute(), "style", "style.css")
styles = open(styles_path, "r").read()
HTML(f"<style>{styles}</style>")

_Note: This notebook must be run locally. Due to how the Swift Simulator operates, this notebook will not run on Google Colab_

# 7.0 Quadratic Rate Motion Control

$\large{\text{Manipulator Differential Kinematics}} \\ \large{\text{Part II: Acceleration and Advanced Applications}}$

$\text{By Jesse Haviland and Peter Corke}$

<br>

The sections of the Tutorial paper related to this notebook are listed next to each contents entry.
It is beneficial to read these sections of the paper before attempting the notebook Section.

### Contents

[7.1 Swift and Robot Setup](#1)

[7.2 Quadratic Rate Motion Control](#2)
* Singularity Escapability
  * Quadratic-Rate Motion Control

[7.3 Position based Servoing with QRMC](#3)

[7.4 QRMC Finishing in a Singularity](#4)

[7.5 QRMC Starting and Finishing in a Singularity](#5)

In [2]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm

# swift is a lightweight browser-based simulator which comes eith the toolbox
from swift import Swift

# spatialgeometry is a utility package for dealing with geometric objects
import spatialgeometry as sg

<br>

<a id='1'></a>

### 7.1 Swift and Robot Setup
---

In this notebook we will be using the Robotics Toolbox for Python and Swift to simulate our motion controllers. Check out Part 1 Notebook 3 for an introduction to these packages.

In [3]:
# Make the environment
env = Swift()

# Launch the simulator, will open a browser tab in your default
# browser (chrome is recommended)
# The realtime flag will ask the simulator to simulate as close as
# possible to realtime as apposed to as fast as possible
env.launch(realtime=True, browser="notebook")

# Make a ur5 robot
robot = rtb.models.UR5()

# Set the joint coordinates to qr
robot.q = robot.qr

# We can then add our robot to the simulator envionment
env.add(robot)

# end-effector axes
ee_axes = sg.Axes(0.1)

# goal axes
goal_axes = sg.Axes(0.1)

# Add the axes to the environment
env.add(ee_axes)
env.add(goal_axes) 

2

<br>

<a id='2'></a>

### 7.2 Quadratic Rate Motion Control
---

Quadratic-rate motion control is a method of controlling a manipulator through or near a singularity. By using the second-order differential kinematics, the controller does not break down at or near a singularity, like the resolved-rate motion controller would. We start by considering the end-effector pose $\bf{x} = f(\bf{q}) \in \mathbb{R}^6$ in terms of the forward kinematics.
Introducing a small change to the joint coordinates $\Delta{\bf{q}}$, we can write

\begin{align*}
    \bf{x} + \Delta{\bf{x}} &= f(\bf{q} + \Delta{\bf{q}}).
\end{align*}

and the Taylor series expansion is

\begin{align*}
    \bf{x} + \Delta{\bf{x}}
    &= f(\bf{q}) +
    \dfrac{\partial f}{\partial \bf{q}} \Delta{\bf{q}} +
    \dfrac{1}{2}
    \left(
        \dfrac{\partial^2 f}{\partial \bf{q}^2} \Delta{\bf{q}}
    \right) \Delta{\bf{q}} +
    \cdots \\
    \Delta{\bf{x}}
    &=
    \bf{J}(\bf{q}) \Delta{\bf{q}} +
    \frac{1}{2}
    \Big(
        \bf{H}(\bf{q}) \Delta{\bf{q}}
    \Big) \Delta{\bf{q}} +
    \cdots
\end{align*}

where for quadratic-rate control we wish to retain both the linear and quadratic terms (the first two terms) of the expansion.

To form our controller, we use an iteration-based Newton-Raphson approach (as we did with resolved-rate motion control), where $k$ indicates the current step. Firstly, we rearrange the above to

\begin{align*}
    \bf{g}(\Delta{\bf{q}}_k)
    &=
    \bf{J}(\bf{q}_k) \Delta{\bf{q}}_k +
    \frac{1}{2}
    \Big(
        \bf{H}(\bf{q}_k) \Delta{\bf{q}}_k
    \Big) \Delta{\bf{q}}_k -
    \Delta{\bf{x}}
\end{align*}

where $\bf{q}_k$ and $\Delta{\bf{q}}_k$ are the manipulator's current joint coordinates and change in joint coordinates respectively, and $\Delta{\bf{x}}$ is the desired change in end-effector position. Taking the derivative of $\bf{g}$ with respect to $\Delta{\bf{q}}$

\begin{align*}
    \dfrac{\partial \bf{g}(\Delta{\bf{q}}_k)}
          {\partial \Delta{\bf{q}}}
    &=
    \bf{J}(\bf{q}_k) + \bf{H}(\bf{q}_k)\Delta{\bf{q}}_k \\
    &= \tilde{\bf{J}}(\bf{q}_k, \Delta{\bf{q}}_k)
\end{align*}

we get a new Jacobian $\tilde{\bf{J}}(\bf{q}_k, \Delta{\bf{q}}_k)$. Using this Jacobian we can create a new linear system

\begin{align*}
    \tilde{\bf{J}}(\bf{q}_k, \Delta{\bf{q}}_k) \bf{\delta}_{\Delta{\bf{q}}} &= \bf{g}(\Delta{\bf{q}}_k) \\
    \bf{\delta}_{\Delta{\bf{q}}} &= \tilde{\bf{J}}(\bf{q}_k, \Delta{\bf{q}}_k)^{-1} \bf{g}(\Delta{\bf{q}}_k)
\end{align*}

where $\bf{\delta}_{\Delta{\bf{q}}}$ is the update to $\Delta{\bf{q}}_k$. The change in joint coordinates at the next step are

\begin{align*}
    \Delta{\bf{q}}_{k+1} = \Delta{\bf{q}}_{k} - \bf{\delta}_{\Delta{\bf{q}}.}
\end{align*}

In the case that $\Delta{\bf{q}} = \bf{0}$, quadratic-rate control reduces to resolved-rate control. This is not suitable if the robot is in or near a singularity. Therefore, a value near $0$ may be used to seed the initial value with $\Delta{\bf{q}} = \bf{0.1}$, or the pseudo-inverse approach could be used

\begin{align*}
    \Delta{\bf{q}} = \bf{J}(\bf{q})^{+} \Delta{\bf{x}}.
\end{align*}

This controller can be used in the same manner as resolved-rate motion control described in Part 1. The manipulator Jacobian and Hessian are calculated in the robot's base frame, and the end-effector velocity $\Delta{\bf{x}}$ can be calculated using the angle-axis method described in Part 1.


Lets make it happen in Python

In [4]:
# Make the environment, launch and add objects
env = Swift()
env.launch(realtime=True, browser="notebook")
env.add(robot)
env.add(ee_axes)

# Change the robot configuration to the ready position
robot.q = np.array([0, -np.pi / 2.2, np.pi / 2, 0.0, -0.0, 0])
robot.qd = np.zeros(robot.n)

# Step the sim to view the robot in this configuration
env.step(0)

# Specify our desired end-effector velocity
ev = [0.05, 0.0, 0.0, 0.0, 0.0, 0.0]

# Specify our timestep
dt = 0.005

# Run the simulation for 4 seconds
for _ in range(400):

    # Work out the manipulator Jacobian, Hessian using the
    # current robot configuration
    J = robot.jacob0(robot.q)
    H = robot.hessian0(robot.q)
    
    # Check if the joint velocity vector norm is close to zero
    # If it is close to 0, use the pseudoinverse technique
    # outlined above
    if np.allclose(np.zeros(robot.n), robot.qd):
        qd = np.linalg.pinv(J) @ ev
    else:
        qd = robot.qd

    # Calculate H * qd
    Hq = np.tensordot(H, qd, axes=(0, 0))

    # Calculate the new Jacobian Jhat
    Jhat = J + Hq

    # Calculate the g vector
    g = J @ qd + 0.5 * Hq @ qd - ev

    # Calculate δq, the change to qd
    δq = np.linalg.inv(Jhat) @ g

    robot.qd = qd - δq

    # Update the ee axes
    ee_axes.T = robot.fkine(robot.q)

    # Step the simulator by dt seconds
    env.step(dt)

In the next example we use the end-effector frames of the Jacobian and Hessian and rotate around the z-axis

In [5]:
# Make the environment, launch and add objects
env = Swift()
env.launch(realtime=True, browser="notebook")
env.add(robot)
env.add(ee_axes)

# Change the robot configuration to the ready position
robot.q = np.array([0, -np.pi / 2, np.pi / 2, 0.0, -0.0, 0])
robot.qd = np.zeros(robot.n)

# Step the sim to view the robot in this configuration
env.step(0)

# Specify our desired end-effector velocity
ev = [0.0, 0.0, 0.0, 0.0, 0.0, 0.5]

# Specify our timestep
dt = 0.005

# Run the simulation for 4 seconds
for _ in range(400):

    # Work out the manipulator Jacobian, Hessian using the
    # current robot configuration
    J = robot.jacobe(robot.q)
    H = robot.hessiane(robot.q)
    
    # Check if the joint velocity vector norm is close to zero
    # If it is close to 0, use the pseudoinverse technique
    # outlined above
    if np.allclose(np.zeros(robot.n), robot.qd):
        qd = np.linalg.pinv(J) @ ev
    else:
        qd = robot.qd

    # Calculate H * qd
    Hq = np.tensordot(H, qd, axes=(0, 0))

    # Calculate the new Jacobian Jhat
    Jhat = J + Hq

    # Calculate the g vector
    g = J @ qd + 0.5 * Hq @ qd - ev

    # Calculate δq, the change to qd
    δq = np.linalg.inv(Jhat) @ g

    robot.qd = qd - δq

    # Update the ee axes
    ee_axes.T = robot.fkine(robot.q)

    # Step the simulator by dt seconds
    env.step(dt)

<br>

<a id='3'></a>

### 7.3 Position based Servoing with QRMC
---

As with previous motion controllers we have developed (Part 1 Notebook 3, Part 2 Notebooks 4, 5) this motion controller is best used with position-based servoing.

In [6]:
# Make the environment, launch and add objects
env = Swift()
env.launch(realtime=True, browser="notebook")
env.add(robot)
env.add(ee_axes)
env.add(goal_axes) 

# Change the robot configuration to the ready position
robot.q = np.array([0, -np.pi / 2, np.pi / 2, 0.0, -0.0, 0])
robot.qd = np.zeros(robot.n)

# Step the sim to view the robot in this configuration
env.step(0)

Tep = (
    robot.fkine(robot.q)
    * sm.SE3.Tx(0.2)
    * sm.SE3.Ty(0.2)
    * sm.SE3.Tz(0.2)
    * sm.SE3.Rx(-np.pi / 2)
    * sm.SE3.Rz(-np.pi / 2)
)
Tep = Tep.A

# Set the goal axes to Tep
goal_axes.T = Tep

# A variable to specify when to break the loop
arrived = False

# Specify our timestep
dt = 0.005

# Specify the gain for the p_servo method
k = np.ones(6)

# Run the simulation until the robot arrives at the goal
while not arrived:

    # The end-effector pose of the panda (using .A to get a numpy array instead of an SE3 object)
    Te = robot.fkine(robot.q).A

    # Calculate the required end-effector velocity and whether the robot has arrived
    ev, arrived = rtb.p_servo(Te, Tep, gain=k, threshold=0.001, method="angle-axis")

    # Work out the manipulator Jacobian, Hessian using the
    # current robot configuration
    J = robot.jacob0(robot.q)
    H = robot.hessian0(robot.q)
    
    # Check if the joint velocity vector norm is close to zero
    # If it is close to 0, use the pseudoinverse technique
    # outlined in Section 7.1
    if np.allclose(np.zeros(robot.n), robot.qd):
        qd = np.linalg.pinv(J) @ ev
    else:
        qd = robot.qd

    # Calculate H * qd
    Hq = np.tensordot(H, qd, axes=(0, 0))

    # Calculate the new Jacobian Jhat
    Jhat = J + Hq

    # Calculate the g vector
    g = J @ qd + 0.5 * Hq @ qd - ev

    # Calculate δq, the change to qd
    δq = np.linalg.inv(Jhat) @ g

    robot.qd = qd - δq

    # Update the ee axes
    ee_axes.T = Te

    # Step the simulator by dt seconds
    env.step(dt)

<br>

<a id='4'></a>

### 7.4 QRMC Finishing in a Singularity
---

In this example the robot must finish its motion at a singularity.

In [7]:
# Make the environment, launch and add objects
env = Swift()
env.launch(realtime=True, browser="notebook")
env.add(robot)
env.add(ee_axes)
env.add(goal_axes) 

# Change the robot configuration to the ready position
robot.q = np.array([0, -np.pi / 2, np.pi / 2, 0.0, -0.0, 0])
robot.qd = np.zeros(robot.n)

# Step the sim to view the robot in this configuration
env.step(0)

Tep = robot.fkine(np.array([np.pi, 0.0, 0.0, 0.0, np.pi/2, 0.0]))
Tep = Tep.A

# Set the goal axes to Tep
goal_axes.T = Tep

# A variable to specify when to break the loop
arrived = False

# Specify our timestep
dt = 0.005

# Specify the gain for the p_servo method
k = np.ones(6)

# Run the simulation until the robot arrives at the goal
while not arrived:

    # The end-effector pose of the panda (using .A to get a numpy array instead of an SE3 object)
    Te = robot.fkine(robot.q).A

    # Calculate the required end-effector velocity and whether the robot has arrived
    ev, arrived = rtb.p_servo(Te, Tep, gain=k, threshold=0.001, method="angle-axis")

    # Work out the manipulator Jacobian, Hessian using the
    # current robot configuration
    J = robot.jacob0(robot.q)
    H = robot.hessian0(robot.q)
    
    # Check if the joint velocity vector norm is close to zero
    # If it is close to 0, use the pseudoinverse technique
    # outlined in Section 7.1
    if np.allclose(np.zeros(robot.n), robot.qd):
        # qd = np.linalg.pinv(J) @ ev
        qd = 0.1 * np.ones(robot.n)
    else:
        qd = robot.qd

    # Calculate H * qd
    Hq = np.tensordot(H, qd, axes=(0, 0))

    # Calculate the new Jacobian Jhat
    Jhat = J + Hq

    # Calculate the g vector
    g = J @ qd + 0.5 * Hq @ qd - ev

    # Calculate δq, the change to qd
    δq = np.linalg.inv(Jhat) @ g

    robot.qd = qd - δq

    # Update the ee axes
    ee_axes.T = Te

    # Step the simulator by dt seconds
    env.step(dt)

<br>

<a id='5'></a>

### 7.5 QRMC Starting and Finishing in a Singularity
---

In this example the robot must start and finish its motion at a singularity.

In [8]:
# Make the environment, launch and add objects
env = Swift()
env.launch(realtime=True, browser="notebook")
env.add(robot)
env.add(ee_axes)
env.add(goal_axes)

# Change the robot configuration
robot.q = np.zeros(robot.n)
robot.qd = np.zeros(robot.n)

# Step the sim to view the robot in this configuration
env.step(0)

Tep = robot.fkine([0, -np.pi / 2, np.pi / 2, 0.0, -0.0, 0])
Tep = Tep.A

# Set the goal axes to Tep
goal_axes.T = Tep

# A variable to specify when to break the loop
arrived = False

# Specify our timestep
dt = 0.005

# Specify the gain for the p_servo method
k = np.ones(6)

# Run the simulation until the robot arrives at the goal
while not arrived:

    # The end-effector pose of the panda (using .A to get a numpy array instead of an SE3 object)
    Te = robot.fkine(robot.q).A

    # Calculate the required end-effector velocity and whether the robot has arrived
    ev, arrived = rtb.p_servo(Te, Tep, gain=k, threshold=0.001, method="angle-axis")

    # Work out the manipulator Jacobian, Hessian using the
    # current robot configuration
    J = robot.jacob0(robot.q)
    H = robot.hessian0(robot.q)
    
    # Check if the joint velocity vector norm is close to zero
    # If it is close to 0, use the pseudoinverse technique
    # outlined in Section 7.1
    if np.allclose(np.zeros(robot.n), robot.qd):
        # qd = np.linalg.pinv(J) @ ev
        qd = 0.1 * np.ones(robot.n)
    else:
        qd = robot.qd

    # Calculate H * qd
    Hq = np.tensordot(H, qd, axes=(0, 0))

    # Calculate the new Jacobian Jhat
    Jhat = J + Hq

    # Calculate the g vector
    g = J @ qd + 0.5 * Hq @ qd - ev

    # Calculate δq, the change to qd
    δq = np.linalg.inv(Jhat) @ g

    robot.qd = qd - δq

    # Update the ee axes
    ee_axes.T = Te

    # Step the simulator by dt seconds
    env.step(dt)